数据模块可细分为4个部分：
> 数据收集：标签和样本
> - 数据划分：训练集、验证集和测试集
> - 数据读取：对应于PyTorch的**DataLoader**。其中DataLoader包括**Sampler**和**DataSet**，Sampler的功能是生成索引，DataSet是根据生成的索引读样本以及标签。
> - 数据预处理：对应于PyTorch的**transforms**

---

## DataLoader与DataSet
`
torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, sampler=None, batch_sampler=None, num_workers=0, collate_fn=None, pin_memory=False, drop_last=False, timeout=0, worker_init_fn=None, multiprocessing_context=None)
`

功能：构建可迭代的数据装载器：
> - dataset: Dataset 类，决定数据从哪里读取以及如何读取
> - batchsize: 批大小
> - num_works:num_works: 是否多进程读取数据
> - sheuffle: 每个 epoch 是否乱序
> - drop_last: 当样本数不能被 batchsize 整除时，是否舍弃最后一批数据

## Epoch, Iteration, Batchsize
> - Epoch: 所有训练样本都已经输入到模型中，称为一个Epoch
> - Iteration: 一批样本输入到模型中，称为一个Iteration
> - Batchsize: 批大小，决定一个iteration有多少样本，也决定了一个Epoch有多少个Iteration

## torch.utils.data.Dataset
功能：Dataset是抽象类，所有自定义的Dataset都需要继承该类，并且重写`__getitem()__`方法和`__len__()`方法。`__getitem()__`方法的作用是接收一个索引，返回索引对应的样本和标签，这是我们自己需要实现的逻辑。`__len__()`方法是返回所有样本的数量。
数据读取包含3个方面:
> - 读取哪些数据：每个 Iteration 读取一个Batchsize大小的数据，每个Iteration应该读取哪些数据。
> - 从哪里读取数据：如何找到硬盘中的数据，应该在哪里设置文件路径参数
> - 如何读取数据：不同的文件需要使用不同的读取方法和库

In [6]:
@staticmethod
def get_img_info(data_dir):
    '''
    实现读取数据的Dataset，编写一个get_img_info()方法，读取每一个图片的路径和对应的标签，组成一个元组，再把所有的元组作为 list 存放到self.data_info变量中，这里需要注意的是标签需要映射到0开始的整数: rmb_label = {"1": 0, "100": 1}
    :param data_dir: 
    :return: 
    '''
    data_info = list()
    for root, dirs, _ in os.walk(data_dir):
        for sub_dir in dirs:
            img_names = os.listdir(os.path.join(root, sub_dir))
            img_names = list(filter(lambda x: x.endswith('.jpg'), img_names))
            for i in range(len(img_names)):
                img_name = img_names[i]
                path_img = os.path.join(root, sub_dir, img_name)
                label = rmb_label[sub_dir]
                data_info.append((path_img, int(label)))
    return data_info        

然后在Dataset的初始化函数中调用`get_img_info()`方法。

In [ ]:
def __init__(self, data_dir, transform=None):
    '''
    rmb面额分类任务的Dataset
    
    :param self: 
    :param data_dir: str, 数据集所在路径
    :param transform: torch.transform，数据预处理
    :return: 
    '''
    # data_info存储所有图片路径和标签，在DataLoader中通过index读取样本
    self.data_info = self.get_img_info(data_dir)
    self.transform = transform

然后在`__getitem__()`方法中根据index 读取`self.data_info`中路径对应的数据，并在这里做transform操作，返回的是样本和标签。

In [ ]:
def __getitem__(self, index):
        # 通过index读取样本
        path_img, label = self.data_info[index]
        # 注意这里需要convert('RGB')
        img = Image.open(path_img).convert('RGB') # 0~255
        if self.transform is not None:
            img = self.transform(img) # 在这里做transform，转为tensor等等
        # 返回是样本和标签
        return img, label

在`__len__()`方法中返回`self.data_info`的长度，即为所有样本的数量。

In [10]:
# 返回所有样本的数量
def __len__(self):
    return len(self.data_info)

在`train_lenet.py`中，分5步构建模型。
第1步设置数据。首先定义训练集、验证集、测试集的路径，定义训练集和测试集的`transforms`。然后构建训练集和验证集的`RMBDataset`对象，把对应的路径和`transforms`传进去。再构建`DataLoder`，设置`batch_size`，其中训练集设置`shuffle=True`，表示每个 Epoch都打乱样本。

In [ ]:
# 构建MyDataset实例
train_data = RMBDataset(data_dr=train_dir, transform=train_transform)
vaild_data = RMBDataset(data_dir=vaild_dir, transform=vaild_transform)

# 构建DataLoder
# 其中训练集设置shuffle=True，表示每个Epoch都打乱样本
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
vaild_loader = DataLoader(dataset=vaild_data, batch_size=BATCH_SIZE)

第2步构建模型，这里采用经典的Lenet图片分类网络。

In [ ]:
net = LeNet(classes=2)
net.initialize_weights()

第3步设置损失函数，这里使用交叉熵损失函数。

In [ ]:
criterion = nn.CrossEntropyLoss()

第4步设置优化器。这里采用SGD优化器。

In [ ]:
# 选择优化器
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9)
 # 设置学习率下降策略
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)    

第5步迭代训练模型，在每一个epoch里面，需要遍历train_loader取出数据，每次取得数据是一个batchsize大小。这里又分为4步。第1步进行前向传播，第2步进行反向传播求导，第3步使用optimizer更新权重，第4步统计训练情况。每一个epoch完成时都需要使用scheduler更新学习率，和计算验证集的准确率、loss。

In [ ]:
for epoch in range(MAX_EPOCH):

    loss_mean = 0.
    correct = 0.
    total = 0.

    net.train()
    # 遍历 train_loader 取数据
    for i, data in enumerate(train_loader):

        # forward
        inputs, labels = data
        outputs = net(inputs)

        # backward
        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()

        # update weights
        optimizer.step()

        # 统计分类情况
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).squeeze().sum().numpy()

        # 打印训练信息
        loss_mean += loss.item()
        train_curve.append(loss.item())
        if (i+1) % log_interval == 0:
            loss_mean = loss_mean / log_interval
            print("Training:Epoch[{:0>3}/{:0>3}] Iteration[{:0>3}/{:0>3}] Loss: {:.4f} Acc:{:.2%}".format(
                epoch, MAX_EPOCH, i+1, len(train_loader), loss_mean, correct / total))
            loss_mean = 0.

    scheduler.step()  # 更新学习率
    # 每个 epoch 计算验证集得准确率和loss
    ...
    ...

我们可以看到每个iteration，我们是从train_loader中取出数据的。

In [ ]:
def __iter__(self):
    if self.num_workers == 0:
        return _SingleProcessDataLoaderIter(self)
    else:
        return _MultiProcessingDataLoaderIter(self)

这里我们没有设置多进程，会执行`_SingleProcessDataLoaderIter`的方法。我们以`_SingleProcessDataLoaderIter`为例。在`_SingleProcessDataLoaderIter`里只有一个方法`_next_data()`，如下：

In [ ]:
def _next_data(self):
    index = self._next_index() # may raise StopIteration
    data = self._dataset_fetcher.fetch(index) # may raise StopIteration
    if self._pin_memory:
        data = _utils.pin_memory.pin_memory(data)
    return data

在该方法中，`self._next_index()`是获取一个batchsize大小的index列表，代码如下：

In [ ]:
def _next_index(self):
    return next(self._sampler_iter)  # may raise StopIteration

其中调用的`sampler`类的`__iter__()`方法返回batch_size大小的随机index列表。

In [ ]:
def __iter__(self):
    batch = []
    for idx in self.sampler:
        batch.append(idx)
        if len(batch) == self.batch_size:
            yield batch
            batch = []
    if len(batch) > 0 and not self.drop_last:
        yield batch

然后再返回看`dataloader`的`_next_data()`方法

In [ ]:
def _next_data(self):
    index = self._next_index()  # may raise StopIteration
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
    if self._pin_memory:
        data = _utils.pin_memory.pin_memory(data)
    return data

在第二行中调用了`self._dataset_fetcher.fetch(index)`获取数据。这里会调用`_MapDatasetFetcher`中的`fetch()`函数：

In [ ]:
def fetch(self, possibly_batched_index):
    if self.auto_collation:
        data = [self.dataset[idx] for idx in possibly_batched_index]
    else:
        data = self.dataset[possibly_batched_index]
    return self.collate_fn(data)

这里调用了`self.dataset[idx]`，这个函数会调用dataset.__getitem__()方法获取具体的数据，所以__getitem__()方法是我们必须实现的。我们拿到的data是一个list，每个元素是一个tunple，每个tunple包括样本和标签。所以最后要使用self.collate_fn(data)把data转换为两个list，第一个元素是样本的batch形式，形状为[16, 3, 32, 32](16是batch size，[3, 32, 32]是图片像素)；第二个元素是标签的batch形式，形状为[16]。
所以在代码中，我们使用`inputs, labels = data`来接收数据。

首先在for循环中遍历`DataLoader`，然后根据是否采用多进程，决定使用单进程或者多进程的`DataLoaderIter`。在DataLoaderIter里调用`Sampler`生成Index的list，再调用`DatasetFetcher`根据index获取数据。在DatasetFetcher里会调用`Dataset`的`__getitem__()`方法获取真正的数据。这里获取的数据是一个list，其中每个元素是`(img, label)`的元组，再使用`collate_fn()`函数整理成一个list，里面包含**两个**元素，分别是img和label的`tenser`。